In [14]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

2.12.0
2.12.0


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [16]:
# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    # Scales each pixel value in the image from a range of 0-255 to 0-1 for normalization
    rescale=1./255,
    # Split the data into training (70%) and validation (30%)
    validation_split=0.3
)

# Validation Data
validation_datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.3
)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    'animal_database/',
    target_size=(256, 256),  # All images will be resized to 150x150
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data
)

# Flow validation images in batches of 32 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    'animal_database/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

class_indices = train_generator.class_indices
print(class_indices)


Found 5782 images belonging to 6 classes.
Found 2473 images belonging to 6 classes.
{'Bear': 0, 'Cat': 1, 'Cow': 2, 'Dog': 3, 'Eagle': 4, 'Gorilla': 5}


In [17]:
# Model Architecture
model = Sequential([
    # First convolutional block
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    # Second convolutional block
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    # Third convolutional block
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    # Fourth convolutional block
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dropout(0.5),

    # Fully connected layers
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping callback
# If the validation accuracy does not improve (increase, since mode='max') for 10 consecutive epochs, the training will stop.
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

# Train your model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=200,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)

Epoch 1/200


2024-03-14 19:11:14.871478: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


180/180 [==============================] - ETA: 0s - loss: 1.7161 - accuracy: 0.4603

2024-03-14 19:14:44.853311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


180/180 [==============================] - 219s 1s/step - loss: 1.7161 - accuracy: 0.4603 - val_loss: 3.8944 - val_accuracy: 0.2013
Epoch 2/200
180/180 [==============================] - 216s 1s/step - loss: 1.2315 - accuracy: 0.5864 - val_loss: 4.6344 - val_accuracy: 0.2037
Epoch 3/200
180/180 [==============================] - 213s 1s/step - loss: 0.9600 - accuracy: 0.6741 - val_loss: 1.7547 - val_accuracy: 0.5300
Epoch 4/200
180/180 [==============================] - 214s 1s/step - loss: 0.8007 - accuracy: 0.7243 - val_loss: 8.5965 - val_accuracy: 0.1843
Epoch 5/200
180/180 [==============================] - 214s 1s/step - loss: 0.6886 - accuracy: 0.7657 - val_loss: 1.3251 - val_accuracy: 0.6351
Epoch 6/200
180/180 [==============================] - 209s 1s/step - loss: 0.6218 - accuracy: 0.7889 - val_loss: 1.9115 - val_accuracy: 0.5921
Epoch 7/200
180/180 [==============================] - 210s 1s/step - loss: 0.5603 - accuracy: 0.8092 - val_loss: 1.7075 - val_accuracy: 0.5584
Epoc

In [ ]:
# Plotting Training Results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(211)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(212)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

In [ ]:
model.save('animals')